In [1]:
import pandas as pd
import string
import re
import datetime


## Marks lists


In [2]:
punctuation_marks = [')','(','>','<',"؛","،",'{','}',"؟",':',"-", '»', '"', '«', '[', ']','"','+','=','?']
marks = ['/','//', '\\','|','!', '%', '&','*','$', '#','؟', '*','.','_' ]
alphabet_string_lower = string.ascii_lowercase
alphabet_string_upper = string.ascii_uppercase
english_char =  list(alphabet_string_lower) + list(alphabet_string_upper)


sep_list = [" ", '\xad', '\u200e','\u200f', '\u200d', '\u200d', '\u200d'] + marks



## Regex  patterns, prefix and postfix list

In [3]:


raw_postfix = [
        ('[\u200c](تر(ین?)?|گری?|های?)' , ''), # تر و ترین و گری به صورت نیم فاصله
        (r'(تر(ین?)?|گری?)(?=$)' , ''), #تر ، ترین، گر، گری در آخر کلمه
        (r'(?<=[^ او])(م|ت|ش|مان|تان|شان)$' , ''), # حذف شناسه های مالکیت و فعل در آخر کلمه
        (r'(ان|ات|گی|گری|های)$' , ''), #ان، ات، ها، های آخر کلمه  
]

raw_arabic_notation = [
    # remove FATHATAN, DAMMATAN, KASRATAN, FATHA, DAMMA, KASRA, SHADDA, SUKUN
    ('[\u064B\u064C\u064D\u064E\u064F\u0650\u0651\u0652]', ''),
    ]

raw_long_letters = [
    (r' +', ' '),  # remove extra spaces
    (r'\n\n+', '\n'),  # remove extra newlines
    (r'[ـ\r]', ''),  # remove keshide, carriage returns
    ]

raw_half_space = [
    ('[\u200c]', ''),
]
 

## verbs, mokassar and morakab list

In [4]:
present_roots =['توان','باش','رو','بر','یاور', 'یانداز', 'یای','یاندیش','بخش','باز','خر','بین','شنو','دار','دان','رسان','شناس','گو','گذار','یاب','لرز','ساز','شو','نویس','خوان','کاه','گیر','خواه','کن' ]

past_roots = ['توانست','بود','کرد','اورد','انداخت','امد','خرید','باخت','برد','رفت','اندیشید','بخشید','دید','شنید','داشت','دانست','رساند','شناخت','گفت','گذشت','یافت','لرزید','ساخت','شد','نوشت','خواند','کاست','گرفت','خواست']
all_verbs_roots = present_roots + past_roots
empty_list = ['','','','','','','']
verb_prefix = ['نمی‌', 'می‌','ن','ب',"" ]
present_verb_postfix = ['م','ی','د','ید','ند','یم']
past_verb_postfix = ['ایم','اید','ای','ام','اند']
past_verb_postfix2 = ['م','ی','ید','ند','یم']
present_verbs = []
past_verbs = []
all_verbs = {}
for pref in verb_prefix:
    for present_root, past_root in zip(present_roots, past_roots):
        for post in past_verb_postfix2:
            all_verbs[pref + past_root+post] = past_root      #نمیرفنم || رفتم
        for post in past_verb_postfix:
            all_verbs[past_root+'ه‌'+post] = past_root        #رفته‌ایم
        for post in present_verb_postfix:
            all_verbs[pref+present_root+post] = present_root  # نمی‌روم
            
words_list = ['ادب', 'آداب', 'طرف', 'اطراف', 'حقیقت', 'حقایق', 'موج', 'امواج', 'ادیب', 'ادبا', 'عمق', 'اعماق', 'خزینه', 'خزائن', 'مرکز', 'مراکز', 'اثر', 'آثار', 'عالم', 'علما', 'خبر', 'اخبار', 'موقع', 'مواقع', 'اسیر', 'اسرا', 'علم', 'علوم', 'دوره', 'ادوار', 'مصرف', 'مصارف', 'اسم', 'اسامی', 'علامت', 'علائم', 'دین', 'ادیان', 'معرفت', 'معارف', 'اسطوره', 'اساطیر', 'علت', 'علل', 'دفتر', 'دفاتر', 'مبحث', 'مباحث', 'امیر', 'امرا', 'عقیده', 'عقائد', 'ذخیره', 'ذخایر', 'ماده', 'مواد', 'امر', 'اوامر', 'عمل', 'اعمال', 'رفیق', 'رفقا', 'مذهب', 'مذاهب', 'امام', 'ائمه', 'عید', 'اعیاد', 'رای', 'آرا', 'مصیبت', 'مصائب', 'اصل', 'اصول', 'عنصر', 'عناصر', 'رسم', 'رسوم', 'معبد', 'معابد', 'افق', 'آفاق', 'عاطفه', 'عواطف', 'رابطه', 'روابط', 'مسجد', 'مساجد', 'بیت', 'ابیات', 'عضو', 'اعضا', 'رمز', 'رموز', 'معبر', 'معابر', 'تاجر', 'تجار', 'عبارت', 'عبارات', 'رجل', 'رجال', 'مظهر', 'مظاهر', 'تصویر', 'تصاویر', 'عجیب', 'عجایب', 'رقم', 'ارقام', 'منظره', 'مناظر', 'جد', 'اجداد', 'فقیه', 'فقها', 'زاویه', 'زوایا', 'مرض', 'امراض', 'جانب', 'جوانب', 'فن', 'فنون', 'زعیم', 'زعما', 'مورد', 'موارد', 'جزیره', 'جزایر', 'فکر', 'افکار', 'سانحه', 'سوانح', 'مرحله', 'مراحل', 'جبل', 'جبال', 'فریضه', 'فرایض', 'سلطان', 'سلاطین', 'مفهوم', 'مفاهیم', 'جریمه', 'جرایم', 'فعل', 'افعال', 'شعر', 'اشعار', 'منبع', 'منابع', 'حادثه', 'حوادث', 'فقیر', 'فقرا', 'شاعر', 'شعرا', 'مکان', 'اماکن', 'حشم', 'احشام', 'قاعده', 'قواعد']
mokassar_dict ={}
for i in range(0,len(words_list),2): 
    mokassar_dict[words_list[i+1]] = words_list[i] # قاعده <= قواعد
                                       

In [5]:
print(len(all_verbs))

1740


## read data

In [5]:

cols = ['id']
df = pd.read_excel('IR_Spring2021_ph12_7k.xlsx', engine = 'openpyxl')
contents = df['content']
doc_ids = df['id']
doc_url = df['url']
docs_num = len(doc_ids)


## tokenizer

In [6]:
def split(txt, seps):
    default_sep = seps[0]    
    # we skip seps[0] because that's the default separator(space) and after tokenize by other we use it.
    for sep in seps[1:]:
        txt = txt.replace(sep, default_sep)
    return [i.strip() for i in txt.split(default_sep)]

#get doc and return (word,doc id)
def tokenizer(text):
    for l in english_char:
        text = text.replace(l,"")
    word_list = split(text.strip().replace("\n"," "), sep_list) # remove newlines and white spaces
    word_list = [x for x in word_list if not x.startswith("ir") and not x.startswith("NID")] # remove hyperlink text and links
    return list(set(word_list))

## normalizer

In [8]:
def remove_punctuation_marks(word_list):
#     word_list = list(word_list)
    for i in range(len(word_list)):
        for mark in punctuation_marks:
            word_list[i] = word_list[i].replace(mark,"")

    return list(set(word_list))


def edit_long_letters(word_list):
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_long_letters]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
    return list(set(word_list))




def remove_arabic_notation(word_list):
    # remove FATHA and likethis
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_arabic_notation]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
    return list(set(word_list))


def create_translation_table(src_list, dst_list):
     return dict((ord(a), b) for a, b in zip(src_list, dst_list)) #map src unicode to dst char

    
def char_digit_Unification(word_list): 
    word_list = [x for x in word_list if not x.isnumeric() or (x.isnumeric() and float(x)<3000)]
    translation_src, translation_dst = ' ىكي“”', ' یکی""' # change arabic form of letterts to persian
    translation_src += 'ئ0123456789أإآ%' # change english number to persian
    translation_dst += 'ی۰۱۲۳۴۵۶۷۸۹ااا٪'
    translations_table = create_translation_table(translation_src, translation_dst)
    for i in range(len(word_list)):
        word_list[i] = word_list[i].translate(translations_table)
    return list(set(word_list))

def remove_mokassar(word_list):
    for i in range(len(word_list)):
        word_list[i] = mokassar_dict.get(word_list[i],word_list[i])
        
    return list(set(word_list))
    


def verb_Steaming(word_list):
    for i in range(len(word_list)):
        word_list[i] = all_verbs.get(word_list[i],word_list[i])
        
    return list(set(word_list))
    

def remove_postfix(word_list):
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_postfix]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            if len(word_list[i])>4 and  not (word_list[i] in all_verbs_roots):
                word_list[i] = pattern.sub(rep, word_list[i])
    return list(set(word_list))


    
    
def remove_prefix(word_list):
    starts =['بی‌' , 'نا', 'با']
    for i in range(len(word_list)):
        for start in starts:
            if word_list[i].startswith(start) and len(word_list[i])>4:
                word_list[i] = word_list[i][len(start):]
                break

    return list(set(word_list))

def morakab_Unification(word_list):
    # remove half-space after all normalization to unify morkab words like گفت‌وگو
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_half_space]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
    return list(set(word_list))

در قسمت نرمالایز ما در ایتدا 
1-علائم نگارشی مثل ویرگول را از کلمات جدا میکنیم
2- حروف کشیده را حدف و تبدیل به حروف عادی میکنیم که فقط با حروف الفبا رو به رو باشیم(در توکنایزر الفبای انگلیسی حذف شده)
3- حال جمع های مکثر را به مفرد تبدیل میکنیم
4-  بعد از حذف جمع مکثر دیگر به علائم آوایی مثل فتحه و ضمه احتیاج نداریم و آنها را حذف میکنیم
5-  حال حروفی که رد عربی و فارسی املای متفاوتی دارند مثل ى ك ي یکسان سازی میکنیم
6-  حال ریشه یابی فعل را بر اساس لیست فعلا های اماده شده انجام میدهیم که ریشه یابی 1740 فعل را میتوانیم انجام دهیم
7- حال به سراغ پسوند ها و پیشوند ها میرویم در اینجا مبنای نیم فاصله برای پیوشند هاست در اصل ما دو دیده برای حفظ ساختار اصلی کلمات داریم که در یکی با استفاده از ریشه یابی فعلها اضافات آن ها گرفته میشود و اگر کلمه ای اآن پسوند و پیشوند هارا داشته باشد حذف ان بلاماتع است
همچنین ما از نیم فاصله برای جدا سازی پیشوند استفاده میکنیم و برای نرمال سازی فاعده نوشتن افعال با نیم فاصله را مدنظر قرار میدهیم 
بدین ترتیب اطلاعات در همگام سازی از بین نمیرود
8- در اخر اگر در وسط کلمه ای نیم فاصله وجود داشته باشید مثل بین‌الملل آن را بعد از تمام ویزایش ها حذف میکنیم تا کلمات مرکب یکسان سازی شوند

In [9]:
def normalizer(word_list):
    word_list = remove_punctuation_marks(word_list)
    word_list = edit_long_letters(word_list)
    word_list = remove_mokassar(word_list)
    word_list = remove_arabic_notation(word_list)
    word_list = char_digit_Unification(word_list)
    word_list = verb_Steaming(word_list)
    word_list = remove_prefix(word_list)
    word_list = remove_postfix(word_list)
    words_list = morakab_Unification(word_list)
    return word_list
    

## create inverted index

In [43]:
        
def create_dictionary(contents, doc_ids):
    for content, id in zip(contents,doc_ids):
        doc_tokens = tokenizer(content)
        doc_tokens = normalizer(doc_tokens)
        term_position_in_doc = 1
        for token in doc_tokens:
            token_postings = inverted_index.get(token,{})
            if len(token_postings) == 0 : #term doesn't exist in dictionary 
                token_postings[id] =[term_position_in_doc]
                
            else:
                plist = token_postings.get(id, [])
                plist.extend(term_position_in_doc)
                token_postings[id] = plist
            term_position_in_doc += 1
            inverted_index[token] = token_postings

            
            
def delete_soptwords(inverted_index):
    stop_words = []
    for key in list(inverted_index):
        if len(inverted_index[key])> 0.25* docs_num or len(key) <2: 
            stop_words.append(key)
            del inverted_index[key]
    return stop_words
    



In [44]:
%%time
inverted_index ={}
create_dictionary(contents[:10], doc_ids[:10])
# print("stop words: ", delete_soptwords(inverted_index))



Wall time: 15 ms


In [45]:
mydict_keys =sorted(inverted_index)
print(len(mydict_keys))

for key in mydict_keys:
    print(key)


0


## check inverted_index

In [13]:
def check_inverted_index():
    word = input("enter a word for checking: ")
    postings_list = inverted_index.get(word, "there isn't such word in inverted_index")
    print(postings_list)

In [14]:
check_inverted_index()

enter a word for checking: سلام
[1, 13, 25, 42, 43, 44, 45, 49, 50, 55, 62, 69, 71, 84, 96, 97, 155, 169, 188, 202, 212, 243, 246, 261, 286, 301, 308, 359, 360, 363, 370, 383, 426, 434, 506, 621, 642, 781, 833, 863, 867, 874, 903, 992, 1003, 1013, 1017, 1068, 1209, 1261, 1276, 1321, 1361, 1494, 1509, 1675, 1684, 1691, 1693, 1721, 1727, 1729, 1738, 1744, 1747, 1749, 1754, 1764, 1765, 1768, 1774, 1779, 1783, 1800, 1808, 1815, 1882, 1904, 1912, 1928, 1933, 1937, 1949, 1950, 1984, 1986, 2022, 2039, 2049, 2055, 2058, 2061, 2069, 2074, 2090, 2094, 2112, 2119, 2146, 2149, 2156, 2157, 2162, 2166, 2175, 2191, 2207, 2216, 2218, 2238, 2248, 2288, 2300, 2301, 2341, 2350, 2352, 2369, 2382, 2387, 2390, 2416, 2422, 2430, 2440, 2446, 2471, 2477, 2497, 2509, 2669, 2680, 2713, 2735, 2754, 2756, 2757, 2763, 2803, 2804, 2813, 2836, 2863, 2866, 2868, 2887, 2896, 2903, 2905, 2933, 2938, 2969, 2990, 2993, 3021, 3044, 3047, 3071, 3078, 3126, 3172, 3174, 3176, 3179, 3184, 3204, 3210, 3227, 3230, 3235, 3239, 32

## response one querry

In [32]:
def one_word_querry():
    word_list =[input("please enter your word")] # we add it into a list because our normalizer input most be  a list
    a = datetime.datetime.now()
    normalized_word = normalizer(word_list)[0]
    print(normalized_word)
    postings_list = inverted_index.get(normalized_word, 0)
    if postings_list == 0:
        print("we coudnt find your word try again")
        return
    b = datetime.datetime.now()
    results = [(x, doc_url[x-1]) for x in postings_list]
    print("{:<5} result in {} ms\nid    -> link\n".format(len(results),1000*(b-a).total_seconds()))
    for doc_id,result in results:
        print( "{:<5} -> {} ".format(doc_id, result))

In [33]:
one_word_querry()

please enter your wordضدعفونی‌کننده
ضدعفونیکننده
17    result in 0.0 ms
id    -> link

3905  -> https://www.isna.ir/news/99081006108/تدابیر-سازمان-فناوری-اطلاعات-ایران-در-کنترل-کرونا 
4155  -> https://www.isna.ir/news/99110301584/آغاز-بیست-وپنجمین-نمایشگاه-نفت-گاز-پالایش-و-پتروشیمی 
5349  -> https://www.isna.ir/news/98120402915/درمان-عفونت-های-زخم-با-نانو-اسپری-محققان-کشور 
5370  -> https://www.isna.ir/news/98121310770/کمبود-ضدعفونی-کننده-در-اقامتگاه-های-عمومی 
5491  -> https://www.isna.ir/news/99012413896/استفاده-زیاد-از-وایتکس-موجب-آسیب-شدید-ریه-می-شود 
5603  -> https://www.isna.ir/news/99022819766/انتقال-انگل-فاسیولا-از-طریق-سبزی-خام-و-آب-چشمه-ها-به-انسان 
5617  -> https://www.isna.ir/news/99030301583/زیارت-کنید-اما-کرونا-نگیرید 
5703  -> https://www.isna.ir/news/99040503875/چگونه-سینما-و-تئاتر-برویم-اما-کرونا-نگیریم 
5711  -> https://www.isna.ir/news/99040503875/چگونه-سینما-و-تئاتر-برویم-اما-کرونا-نگیریم 
5714  -> https://www.isna.ir/news/99040705025/بیمار-۷۳-ساله-کرونایی-مرخص-شد-س

In [26]:
def multi_word_query():
    query = input("enter ypur querry: ")
    a = datetime.datetime.now()
    words_list = tokenizer(query)
    words_list = normalizer(words_list)
    print(words_list)
    docs_score = {}
    for word in words_list:
        if word in inverted_index.keys():
            for doc_id in inverted_index[word]:
                if doc_id not in docs_score.keys():
                    docs_score[doc_id] = 1
                else:
                    docs_score[doc_id] += 1

    docs_score = sorted(docs_score.items(), key=lambda kv: kv[1], reverse=True)
    b = datetime.datetime.now()
    results = [(x, doc_url[x-1]) for x,_ in docs_score]
    print("{:<5} result in {} ms\nid    -> link\n".format(len(results), 1000*(b-a).total_seconds()))
    for doc_id,result in results:
        print( "{:<5} -> {} ".format(doc_id, result))



In [39]:
multi_word_query()

enter ypur querry: عصاره فضایل ملت
['عصاره', 'فضایل', 'ملت']
224   result in 1.008 ms
id    -> link

6727  -> https://www.isna.ir/news/98050200866/واکنش-اینستاگرامی-رییس-اسبق-سازمان-غذا-و-دارو-به-ادعای-دو-تابعیتی 
6730  -> https://www.isna.ir/news/98050200866/واکنش-اینستاگرامی-رییس-اسبق-سازمان-غذا-و-دارو-به-ادعای-دو-تابعیتی 
3156  -> https://www.isna.ir/news/98112216539/فروزنده-اقتصاد-دانش-بنیان-و-اقتصاد-مقاومتی-راه-نجات-کشور-است 
1738  -> https://www.isna.ir/news/99011105670/بیانیه-جهاددانشگاهی-به-مناسبت-فرا-رسیدن-روز-جمهوری-اسلامی 
2240  -> https://www.isna.ir/news/99081610207/سیاست-جمهوری-اسلامی-ایران-هیچگاه-با-تغییر-رؤسای-جمهور-در-آمریکا 
5537  -> https://www.isna.ir/news/99021007188/تشکیل-کمیته-ملی-زالوی-طبی-از-سوی-ستاد-گیاهان-دارویی 
6671  -> https://www.isna.ir/news/98032210328/تولید-کرمی-برای-درمان-ترک-های-پوستی-با-گیاه-آب-بشقابی 
6673  -> https://www.isna.ir/news/98032210328/تولید-کرمی-برای-درمان-ترک-های-پوستی-با-گیاه-آب-بشقابی 
6743  -> https://www.isna.ir/news/98051306482/قه